# Nuevos Objetivos de Recomendaciones para Quilmes

## Descripción
Este proyecto tiene como objetivo ajustar las recomendaciones de productos de Quilmes para cumplir con ciertas restricciones y objetivos de negocio. Utiliza datos de clientes, productos y recomendaciones previas para redistribuir las recomendaciones de manera que se adhieran a los criterios especificados.

## Instrucciones de Uso
1. **Carga de Datos**: El script comienza cargando los datos necesarios de tres archivos Excel: clientes del último mes (`customers_last_month.xlsx`), recomendaciones actuales (`recommendations.xlsx`) y detalles de productos (`products.xlsx`).

2. **Preparación de Datos**:
   - Se calcula el total de recomendaciones por producto (`product_desc`).
   - Se combina esta información con el número de clientes del último mes y se calcula el ratio de recomendaciones por cliente.
   - Se agrega la información del segmento de marca (Premium o no) de cada producto.

3. **Algoritmo de Ajuste de Recomendaciones**:
   - Se asegura que "Brahma Dorada-1 Lts-Bottle-RET" tenga al menos 200 recomendaciones.
   - Se ajusta para que al menos el 20% del total de las recomendaciones sean de productos Premium.
   - Se ajustan las recomendaciones para que estén entre el 75% y el 200% del número de clientes del último mes.

4. **Análisis de los Resultados**:
   - Se ordenan los productos según la variación en el número de recomendaciones comparado con el número original.
   - Se exporta este análisis a un archivo CSV para su revisión y uso futuro.

5. **Cálculo de Recomendaciones por Marca**:
   - Se calcula el número total de recomendaciones por marca y se presenta en orden descendente.

## Estructura del Código
El código se organiza en bloques funcionales que realizan tareas específicas, desde la carga de datos hasta la generación de análisis y reportes. 

## Personalización
Para adaptar el script a otros conjuntos de datos o criterios de ajuste, modifique los parámetros y rutas de archivo en la sección de carga de datos y los criterios específicos en la función `ajuste_recomendaciones`.

## Contribuciones
Las contribuciones a este proyecto son bienvenidas. Por favor, siga las mejores prácticas de codificación y documentación al hacer modificaciones o adiciones.


In [1]:
# Importamos las librerías necesarias
import pandas as pd
from typing import Tuple

# Cargamos los datasets
clientes_ultimo_mes_df = pd.read_excel("customers_last_month.xlsx")
recomendaciones_df = pd.read_excel("recommendations.xlsx")
productos_df = pd.read_excel("products.xlsx")

In [2]:
# Preparamos los datos
# Calculamos las recomendaciones por product_desc
recomendaciones_por_producto = recomendaciones_df.groupby("product_desc").size().reset_index(name='recomendaciones_total')

# Combinamos los dataframes y calculamos el ratio
analisis_recomendaciones_df = pd.merge(recomendaciones_por_producto, clientes_ultimo_mes_df, on="product_desc", how="left")
analisis_recomendaciones_df['ratio_recomendaciones'] = analisis_recomendaciones_df['recomendaciones_total'] / analisis_recomendaciones_df['customers_last_month']

# Agregamos el segmento de producto y limpiamos
analisis_recomendaciones_df = pd.merge(analisis_recomendaciones_df, productos_df[['product_desc', 'brand_segment_I']], on="product_desc", how="left")
analisis_recomendaciones_df.drop_duplicates(inplace=True)

# Reclaculamos las recomendaciones
recomendaciones_total = analisis_recomendaciones_df['recomendaciones_total'].sum()

# Creamos una función con nuestro algortimo de ajuste
def ajuste_recomendaciones(df: pd.DataFrame, recomendaciones_total: int, premium_ratio: float, producto_especifico: str, min_producto_especifico_rec: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Hacemos una copia del dataframe original
    df_ajustado = df.copy()
    # Paso 1: Nos aseguramos que el producto "Brahma Dorada-1 Lts-Bottle-RET" tenga al menos 200 recomendaciones
    producto_especifico_rec = df_ajustado.loc[df_ajustado['product_desc'] == producto_especifico, 'recomendaciones_total'].iloc[0]
    if producto_especifico_rec < min_producto_especifico_rec:
        ajuste = min_producto_especifico_rec - producto_especifico_rec
        df_ajustado.loc[df_ajustado['product_desc'] == producto_especifico, 'recomendaciones_total'] += ajuste
        recomendaciones_total -= ajuste

    # Paso 2: Nos aseguramos que al menos 20% de las recomendaciones son para productos premium
    recomendaciones_premium = df_ajustado[df_ajustado['brand_segment_I'] == 'Premium']['recomendaciones_total'].sum()
    while recomendaciones_premium / recomendaciones_total < premium_ratio:
        productos_premium = df_ajustado[df_ajustado['brand_segment_I'] == 'Premium']
        for _, row in productos_premium.iterrows():
            if recomendaciones_total > 0:
                df_ajustado.loc[df_ajustado['product_desc'] == row['product_desc'], 'recomendaciones_total'] += 1
                recomendaciones_premium += 1
                recomendaciones_total -= 1
            else:
                break

    # Paso 3: Ajustamos recommendaciones entre 75% y 200% 
    # Disminuímos recommendaciones sobre el 200%
    sobre_recomendados = df_ajustado[df_ajustado['ratio_recomendaciones'] > 2]
    for _, row in sobre_recomendados.iterrows():
        recomendaciones_objetivo = int(row['customers_last_month'] * 2)  # Target is 200% of last month's customers
        if row['recomendaciones_total'] > recomendaciones_objetivo:
            ajuste = row['recomendaciones_total'] - recomendaciones_objetivo
            df_ajustado.loc[df_ajustado['product_desc'] == row['product_desc'], 'recomendaciones_total'] -= ajuste
            recomendaciones_total += ajuste

    # Incrementamos recommendaciones por debajo del 75%
    bajo_recomendados = df_ajustado[df_ajustado['ratio_recomendaciones'] < 0.75]
    for _, row in bajo_recomendados.iterrows():
        if recomendaciones_total > 0:
            recomendaciones_objetivo = int(row['customers_last_month'] * 0.75)  # Target is 75% of last month's customers
            if row['recomendaciones_total'] < recomendaciones_objetivo:
                ajuste = recomendaciones_objetivo - row['recomendaciones_total']
                df_ajustado.loc[df_ajustado['product_desc'] == row['product_desc'], 'recomendaciones_total'] += min(ajuste, recomendaciones_total)
                recomendaciones_total -= min(ajuste, recomendaciones_total)


    # Recalculamos el ratio y la variación
    df_ajustado['nuevo_ratio'] = df_ajustado['recomendaciones_total'] / df_ajustado['customers_last_month']
    df_ajustado['variacion_del_original'] = df_ajustado['recomendaciones_total'] - df['recomendaciones_total']

    # Devolvemos el dataframe ajustado
    return df_ajustado, df_ajustado[['product_desc', 'recomendaciones_total', 'customers_last_month', 'nuevo_ratio', 'variacion_del_original']]

# Aplicamos la función con el algoritmo de ajuste
recomendaciones_ajustadas_df, analisis_recomendaciones_df = ajuste_recomendaciones(
    analisis_recomendaciones_df,
    recomendaciones_total,
    0.20,  # 20% producto premium
    "Brahma Dorada-1 Lts-Bottle-RET", # Producto bandera de la empresa
    200  # Mínimo de recomendaciones para producto bandera
)

In [3]:
# Ordenamos los resultados por la columna 'variación del original' y mostramos
analisis_recomendaciones_df = analisis_recomendaciones_df.sort_values(by='variacion_del_original', ascending=False)
analisis_recomendaciones_df.head()

,product_desc,recomendaciones_total,customers_last_month,nuevo_ratio,variacion_del_original
148,Brahma Dorada-1 Lts-Bottle-RET,200,195.0,1.025641,150
95,Andes Origen Roja-1/3 Lts-Bottle-RET,123,164.0,0.750000,95
928,Quilmes Ipa-1/2 Lts-Cans,122,163.0,0.748466,78
0,1890-1 Lts-Bottle-RET,121,162.0,0.746914,73
914,Quilmes Doble-1/2 Lts-Cans,59,79.0,0.746835,55


In [4]:
# Exportamos el análisis a un .csv
analisis_recomendaciones_df.to_csv('recomendaciones_ajustadas.csv', index=False)

In [5]:
# Calculamos el número de recomendaciones por producto
# Relacionamos los dataframes necesarios
recomendaciones_por_marca_df = pd.merge(recomendaciones_ajustadas_df[['product_desc', 'recomendaciones_total']], 
                                    productos_df[['product_desc', 'brand']], 
                                    on='product_desc', 
                                    how='left')

# Sumamos las recomendaciones por marca
recomendaciones_por_marca = recomendaciones_por_marca_df.groupby('brand')['recomendaciones_total'].sum().reset_index(name='conteo_recomendaciones')

# Ordenamos y mostramos
recomendaciones_por_marca = recomendaciones_por_marca.sort_values(by='conteo_recomendaciones', ascending=False)
recomendaciones_por_marca.head()

,brand,conteo_recomendaciones
11,Brahma,31745
33,Quilmes Clasica,30173
39,Stella artois,14633
13,Budweiser,12306
14,Corona,6455
